In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
import pandas as pd
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from numpy import dot
from numpy.linalg import norm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [2]:
#匯入user.movie互動資料
rating = pd.read_csv("Movielens\\user_movie.dat",sep = "\t",names=["userId","movieId","rating","timestamp"])
print("互動資料:")
rating

互動資料:


,userId,movieId,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [3]:
User = pd.read_csv("Movielens\\user_user(knn).dat",sep = "\t",names=["auserId","buserId","similarity"])
print("user互動similarity:")
User

user互動similarity:


,auserId,buserId,similarity
0,1,823,0.340297
1,1,92,0.328166
2,1,339,0.327473
3,1,297,0.322998
4,1,806,0.318959
...,...,...,...
47145,943,311,0.292593
47146,943,881,0.292412
47147,943,64,0.291997
47148,943,246,0.291895


In [4]:
User = pd.read_csv("Movielens\\movie_movie(knn).dat",sep = "\t",names=["auserId","buserId","similarity"])
print("user互動similarity:")
User

user互動similarity:


,auserId,buserId,similarity
0,1,151,0.321301
1,1,121,0.306714
2,1,222,0.304458
3,1,742,0.301808
4,1,117,0.287419
...,...,...,...
82793,1682,80,0.140691
82794,1682,369,0.140691
82795,1682,158,0.139170
82796,1682,741,0.139170


In [5]:
#建立user-item矩陣
user_item = pd.pivot_table(rating,values='rating',columns='movieId',index='userId')
user_item

movieId,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
userId,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
user_item=user_item.values
user_item = np.where(np.isnan(user_item), 0, user_item)
user_item

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [7]:
#分割資料集
train_data, test_data = train_test_split(user_item, random_state=None, train_size=0.8)
print("Train data:")
train_data

Train data:


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [8]:
print("Test data:")
test_data

Test data:


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [9]:
# Define the number of folds (K) for cross-validation
k = 5

# Define lists to store the evaluation metrics for each fold
rmse_scores = []
mae_scores = []

# Perform K-fold Cross-Validation
kf = KFold(n_splits=k)

In [19]:
for train_index, test_index in kf.split(train_data):
    # Split the data into train and test sets for the current fold
    train_ratings, test_ratings = user_item[train_index], user_item[test_index]

    # Data preprocessing (do normalized)
    normalized_Train_data = train_ratings / np.max(train_ratings)
    normalized_Test_data = test_ratings / np.max(test_ratings)
    
    # Define and compile your model
    model = Sequential()
    model.add(Dense(400, activation='relu', input_dim=user_item.shape[1]))
    model.add(Dropout(0.3))
    model.add(Dense(200, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(user_item.shape[1], activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.summary()


    # Train the model
    model.fit(normalized_Train_data, normalized_Train_data, epochs=30, batch_size=16, verbose=1)

    # Make predictions on the test set
    predicted_ratings = model.predict(normalized_Test_data)
    print(predicted_ratings)

    # Calculate evaluation metrics (RMSE and MAE)
    true_ratings = test_ratings.flatten()
    predicted_ratings = predicted_ratings.flatten()

    rmse = np.sqrt(mean_squared_error(true_ratings, predicted_ratings))
    mae = np.mean(np.abs(true_ratings - predicted_ratings))

    # Append the evaluation scores to the lists
    rmse_scores.append(rmse)
    mae_scores.append(mae)

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 400)               673200    
                                                                 
 dropout_20 (Dropout)        (None, 400)               0         
                                                                 
 dense_31 (Dense)            (None, 200)               80200     
                                                                 
 dropout_21 (Dropout)        (None, 200)               0         
                                                                 
 dense_32 (Dense)            (None, 1682)              338082    
                                                                 
Total params: 1,091,482
Trainable params: 1,091,482
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
38/38 [=========================

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_36 (Dense)            (None, 400)               673200    
                                                                 
 dropout_24 (Dropout)        (None, 400)               0         
                                                                 
 dense_37 (Dense)            (None, 200)               80200     
                                                                 
 dropout_25 (Dropout)        (None, 200)               0         
                                                                 
 dense_38 (Dense)            (None, 1682)              338082    
                                                                 
Total params: 1,091,482
Trainable params: 1,091,482
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
38/38 [=========================

_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_42 (Dense)            (None, 400)               673200    
                                                                 
 dropout_28 (Dropout)        (None, 400)               0         
                                                                 
 dense_43 (Dense)            (None, 200)               80200     
                                                                 
 dropout_29 (Dropout)        (None, 200)               0         
                                                                 
 dense_44 (Dense)            (None, 1682)              338082    
                                                                 
Total params: 1,091,482
Trainable params: 1,091,482
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
38/38 [==============================] - 1s 6ms/step - 

In [28]:
for train_index, test_index in kf.split(train_data):
    # Split the data into train and test sets for the current fold
    train_ratings, test_ratings = user_item[train_index], user_item[test_index]

    # Data preprocessing (do normalized)
    normalized_Train_data = train_ratings / np.max(train_ratings)
    normalized_Test_data = test_ratings / np.max(test_ratings)
    
    # Define and compile your model
    model = Sequential()
    model.add(Dense(16, activation='relu', input_dim=user_item.shape[1]))
    model.add(Dropout(0.3))
    model.add(Dense(8, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(user_item.shape[1], activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.summary()


    # Train the model
    model.fit(normalized_Train_data, normalized_Train_data, epochs=30, batch_size=16, verbose=1)

    # Make predictions on the test set
    predicted_ratings = model.predict(normalized_Test_data)
    print(predicted_ratings)

    # Calculate evaluation metrics (RMSE and MAE)
    true_ratings = test_ratings.flatten()
    predicted_ratings = predicted_ratings.flatten()

    rmse = np.sqrt(mean_squared_error(true_ratings, predicted_ratings))
    mae = np.mean(np.abs(true_ratings - predicted_ratings))

    # Append the evaluation scores to the lists
    rmse_scores.append(rmse)
    mae_scores.append(mae)

Model: "sequential_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_105 (Dense)           (None, 16)                26928     
                                                                 
 dropout_60 (Dropout)        (None, 16)                0         
                                                                 
 dense_106 (Dense)           (None, 8)                 136       
                                                                 
 dropout_61 (Dropout)        (None, 8)                 0         
                                                                 
 dense_107 (Dense)           (None, 1682)              15138     
                                                                 
Total params: 42,202
Trainable params: 42,202
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
38/38 [==============================]

Model: "sequential_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_111 (Dense)           (None, 16)                26928     
                                                                 
 dropout_64 (Dropout)        (None, 16)                0         
                                                                 
 dense_112 (Dense)           (None, 8)                 136       
                                                                 
 dropout_65 (Dropout)        (None, 8)                 0         
                                                                 
 dense_113 (Dense)           (None, 1682)              15138     
                                                                 
Total params: 42,202
Trainable params: 42,202
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
38/38 [==============================]

_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_117 (Dense)           (None, 16)                26928     
                                                                 
 dropout_68 (Dropout)        (None, 16)                0         
                                                                 
 dense_118 (Dense)           (None, 8)                 136       
                                                                 
 dropout_69 (Dropout)        (None, 8)                 0         
                                                                 
 dense_119 (Dense)           (None, 1682)              15138     
                                                                 
Total params: 42,202
Trainable params: 42,202
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
38/38 [==============================] - 0s 2ms/step - loss: 

In [29]:
# Calculate and print the average evaluation scores
avg_rmse = np.mean(rmse_scores)
# avg_mae = np.mean(mae_scores)
print("Average RMSE:", avg_rmse)
# print("Average MAE:", avg_mae)

Average RMSE: 0.8848075568105578


In [12]:
predicted_ratings = model.predict(test_data)
predicted_ratings

6/6 [==============================] - 0s 3ms/step


array([[ 7.4782300e-01,  4.5401178e-02, -7.9210941e-03, ...,
        -1.0471557e-02,  1.4117311e-02,  7.8749312e-03],
       [-4.6319932e-01, -8.4114015e-02, -4.9162246e-02, ...,
        -3.7840093e-03,  1.5474216e-05, -6.4573600e-03],
       [ 7.6942241e-01,  1.2825084e-01,  1.9776590e-01, ...,
         1.3136594e-04, -5.7781804e-03, -1.1077246e-02],
       ...,
       [ 1.2242266e+00, -1.3858713e-01,  1.4246212e-01, ...,
        -9.8262320e-04,  2.0492013e-04, -4.7205738e-03],
       [ 3.0577534e-01, -1.6062733e-02,  8.8236146e-03, ...,
        -7.9621896e-05, -1.3419022e-04, -4.6160645e-03],
       [ 1.5414940e+00, -1.5390078e-03, -7.7480182e-02, ...,
        -4.2775874e-03,  7.1041249e-03,  1.4999604e-02]], dtype=float32)

In [13]:
print(len(predicted_ratings[0]))
print( len(test_data[0]))

1682
1682


In [14]:
# A=[]
# A=predicted_ratings[1]
print(type(predicted_ratings))
predicted_ratings

<class 'numpy.ndarray'>


array([[ 7.4782300e-01,  4.5401178e-02, -7.9210941e-03, ...,
        -1.0471557e-02,  1.4117311e-02,  7.8749312e-03],
       [-4.6319932e-01, -8.4114015e-02, -4.9162246e-02, ...,
        -3.7840093e-03,  1.5474216e-05, -6.4573600e-03],
       [ 7.6942241e-01,  1.2825084e-01,  1.9776590e-01, ...,
         1.3136594e-04, -5.7781804e-03, -1.1077246e-02],
       ...,
       [ 1.2242266e+00, -1.3858713e-01,  1.4246212e-01, ...,
        -9.8262320e-04,  2.0492013e-04, -4.7205738e-03],
       [ 3.0577534e-01, -1.6062733e-02,  8.8236146e-03, ...,
        -7.9621896e-05, -1.3419022e-04, -4.6160645e-03],
       [ 1.5414940e+00, -1.5390078e-03, -7.7480182e-02, ...,
        -4.2775874e-03,  7.1041249e-03,  1.4999604e-02]], dtype=float32)

In [15]:
from sklearn.metrics import ndcg_score
k = 10  # Top k 推薦項目數量

ndcg_scores = []

for i in range(len(test_data)):
    true_labels = test_data[i]
    predicted_labels = predicted_ratings[i]

    # 按照預測評分降續排序，提取前k個項目的真實評分
    top_k_indices = np.argsort(predicted_labels)[::-1][:k]
    true_ratings_k = true_labels[top_k_indices]

    # 計算累積增益
    cumulative_gain = (2 ** true_ratings_k - 1) / np.log2(np.arange(2, k + 2))

    # 按照真實評分降續排序，提取前k個項目的真實評分
    ideal_true_ratings_k = np.sort(true_labels)[::-1][:k]

    # 計算理想累積增益
    ideal_cumulative_gain = (2 ** ideal_true_ratings_k - 1) / np.log2(np.arange(2, k + 2))

    # 計算NDCG@k
    ndcg_at_k = np.sum(cumulative_gain) / np.sum(ideal_cumulative_gain)

    ndcg_scores.append(ndcg_at_k)

# 計算平均NDCG@k
average_ndcg_at_k = np.mean(ndcg_scores)

print("Average NDCG@{}: {:.4f}".format(k, average_ndcg_at_k))

Average NDCG@10: 0.6411


In [16]:
min_rating = 0
max_rating = 5
k = 10  # Top k 推薦項目數量
threshold = 0.5  

recall_scores = []

for i in range(len(test_data)):
    true_ratings = test_data[i]
    predicted_values = predicted_ratings[i] * (max_rating - min_rating) + min_rating

    # 預測預測評分中前k個最高評分的項目索引
    top_k_indices = predicted_values.argsort()[-k:][::-1]
#     print("預測的movies:")
#     print(top_k_indices)

    # 計算真實評分中大於等於threshold的項目數量
    relevant_items = np.sum(true_ratings >= threshold)

    # 計算推薦項目中真實評分大於等於threshold的項目數量
    recommended_relevant_items = np.sum(true_ratings[top_k_indices] >= threshold)

    # 計算 Recall@k
    recall_at_k = recommended_relevant_items / relevant_items

    recall_scores.append(recall_at_k)

# 計算平均 Recall@k
average_recall_at_k = np.mean(recall_scores)

print("Average Recall@{}: {:.4f}".format(k, average_recall_at_k))


Average Recall@10: 0.1632
